In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import *
spark = SparkSession.builder \
    .config('spark.driver.memory', '3g') \
    .enableHiveSupport().getOrCreate()

In [6]:
! ls /data

Badges	Comments  PostLinks  Posts  Tags  Users  Votes


In [9]:
base_path = 'file:///data'

In [12]:
posts = spark.read.format('parquet').load(f'{base_path}/Posts')

In [13]:
posts.count()

45919817

In [14]:
posts.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- ParentId: integer (nullable = true)
 |-- PostTypeId: integer (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- Score: integer (nullable = true)
 |-- ViewCount: integer (nullable = true)
 |-- Body: string (nullable = true)
 |-- OwnerUserId: integer (nullable = true)
 |-- LastActivityDate: timestamp (nullable = true)
 |-- Title: string (nullable = true)
 |-- Tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- CommentCount: integer (nullable = true)
 |-- AnswerCount: integer (nullable = true)
 |-- LastEditDate: timestamp (nullable = true)
 |-- LastEditorUserId: integer (nullable = true)
 |-- AcceptedAnswerId: integer (nullable = true)
 |-- FavoriteCount: integer (nullable = true)
 |-- OwnerDisplayName: string (nullable = true)
 |-- ClosedDate: timestamp (nullable = true)
 |-- LastEditorDisplayName: string (nullable = true)
 |-- CommunityOwnedDate: timestamp (nullable = true)



In [15]:
posts.groupBy('PostTypeId').count().show()

+----------+--------+
|PostTypeId|   count|
+----------+--------+
|         1|18154493|
|         6|     324|
|         3|     167|
|         5|   49909|
|         4|   49909|
|         8|       2|
|         7|       4|
|         2|27665009|
+----------+--------+



In [16]:
QUESTION_POST_TYPE = 1
questions = posts.where(col('PostTypeId') == QUESTION_POST_TYPE)

In [18]:
questions.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- ParentId: integer (nullable = true)
 |-- PostTypeId: integer (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- Score: integer (nullable = true)
 |-- ViewCount: integer (nullable = true)
 |-- Body: string (nullable = true)
 |-- OwnerUserId: integer (nullable = true)
 |-- LastActivityDate: timestamp (nullable = true)
 |-- Title: string (nullable = true)
 |-- Tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- CommentCount: integer (nullable = true)
 |-- AnswerCount: integer (nullable = true)
 |-- LastEditDate: timestamp (nullable = true)
 |-- LastEditorUserId: integer (nullable = true)
 |-- AcceptedAnswerId: integer (nullable = true)
 |-- FavoriteCount: integer (nullable = true)
 |-- OwnerDisplayName: string (nullable = true)
 |-- ClosedDate: timestamp (nullable = true)
 |-- LastEditorDisplayName: string (nullable = true)
 |-- CommunityOwnedDate: timestamp (nullable = true)



### 1. Czy pytanie może być community-owned? (`CommunityOwnedDate`)

In [20]:
questions.where(col('CommunityOwnedDate').isNotNull()).select('Id').first()

Row(Id=839899)

### 2. Przykładowy ID zamkniętego pytania z września 2019 (`ClosedDate`)

https://stackoverflow.com/q/839899

In [22]:
questions.where(year('ClosedDate') == 2019) \
    .where(month('ClosedDate') == 9).select('Id').show()

+--------+
|      Id|
+--------+
|  206258|
| 2279993|
|23043938|
|53167196|
|57614714|
|57649465|
|57689162|
|57029989|
|57576987|
|47334139|
|40689858|
|57722308|
|57726456|
|57727515|
|57728075|
|57730003|
|57733132|
|57734725|
|57734751|
|57736739|
+--------+
only showing top 20 rows

